## Chi squared tests for 2xn tables and posthoc analysis 

Cisneros_Villanueva et al, 2022.

By: Magdalena Ríos Romero, coauthor.

The objective of this code is to perform chi squared tests with posthoc analyses for 2Xn tables (tables that contain variables with more than 2 levels). In these cases, we need to perform posthoc and multiple testing correction analysis. Using two cusom functions, we can perform chi square calculations, posthoc analysis and corrections.   

In [1]:
import pandas as pd 

In [2]:
import numpy as np


I will start the analysis with the TCGA cohort.

In [103]:
base_TCGA=pd.read_csv('baseTCGA.csv')

In [111]:
base_TCGA

,Diagnosis Age,Age,Clinical stage,Clinical Stage,Histopathology type,Histopathology,Histopathology.1,Radiation Therapy,Radiation Therapy.1,Subtype PAM50,PAM50,SOX9-AS1,SOX9-AS1_expression,Unnamed: 13
0,67,2,STAGE IA,1,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,No,2,BRCA_Basal,4,"7,887",High,1
1,72,2,STAGE IIA,2,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,No,2,BRCA_Basal,4,"7,728",High,1
2,51,2,STAGE IIB,2,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,Yes,1,BRCA_Basal,4,"7,64",High,1
3,59,2,STAGE IA,1,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,No,2,BRCA_Basal,4,"7,62",High,1
4,47,1,STAGE I,1,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,Yes,1,NaN,5,"7,433",High,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,61,2,STAGE I,1,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,Yes,1,BRCA_LumA,1,"-9,966",Low,2
1062,74,2,STAGE IIA,2,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,No,2,BRCA_LumA,1,"-9,966",Low,2
1063,75,2,STAGE IIIA,3,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,No,2,BRCA_LumA,1,"-9,966",Low,2
1064,76,2,STAGE IIA,2,Breast Invasive Ductal Carcinoma,Infiltrating Ductal Carcinoma,1,No,2,BRCA_LumB,2,"-9,966",Low,2


In [7]:
base_TCGA.columns

Index(['Diagnosis Age', 'Age', 'Clinical stage', 'Clinical Stage',
       'Histopathology type', 'Histopathology', 'Histopathology.1',
       'Radiation Therapy', 'Radiation Therapy.1', 'Subtype PAM50', 'PAM50',
       'SOX9-AS1', 'SOX9-AS1_expression', 'Unnamed: 13'],
      dtype='object')

In [112]:
base_TCGA['Age'].isnull().sum()

0

In [105]:
base_TCGA['Histopathology'].isnull().sum()

0

In [106]:
base_TCGA['Subtype PAM50'].isnull().sum()

123

In [107]:
base_TCGA['Radiation Therapy'].isnull().sum()

100

In [108]:
base_TCGA['SOX9-AS1_expression'].isnull().sum()

0

## Counting frequencies

I will count frequencies for each category. These frequencies will include NANs, as we are reporting NANs as a separate category. Of note, NANs will be ignored in the downstream chi square analysis. 

In [110]:
#Without NANs 
pd.crosstab(base_TCGA['Age'],base_TCGA['SOX9-AS1_expression'])


SOX9-AS1_expression,High,Low
Age,,
1,161,128
2,372,405


In [113]:
#Without NANs 
pd.crosstab(base_TCGA['Histopathology'],base_TCGA['SOX9-AS1_expression'])

SOX9-AS1_expression,High,Low
Histopathology,,
Infiltrating Ductal Carcinoma,372,387
Infiltrating Lobular Carcinoma,116,85
Mixed Histology (NOS),12,17
Other,33,44


In [98]:
#With NANs: passing the fillna function
pd.crosstab(base_TCGA['Radiation Therapy'].fillna('missing'),base_TCGA['SOX9-AS1_expression'])

SOX9-AS1_expression,High,Low
Radiation Therapy,,
No,207,217
Yes,270,272
missing,56,44


In [115]:
#With NANs 
pd.crosstab(base_TCGA['Subtype PAM50'].fillna('missing'),base_TCGA['SOX9-AS1_expression'])

SOX9-AS1_expression,High,Low
Subtype PAM50,,
BRCA_Basal,163,8
BRCA_Her2,26,51
BRCA_LumA,233,265
BRCA_LumB,35,162
missing,76,47


In [114]:
#With NANs encoded as category number 5
pd.crosstab(base_TCGA['Clinical Stage'],base_TCGA['SOX9-AS1_expression'])


SOX9-AS1_expression,High,Low
Clinical Stage,,
1,91,89
2,313,291
3,112,133
4,9,9
5,8,11


Making crosstabs with the variables of interest in order to feed the chi squared test with posthoc function below.
Of note, the crosstab function in pandas does not include NANs as a category in its default parameters. It simply ignores NANs when constructing the crosstab. Taking advantage of this, I will skip the drop NANs step. 

In [142]:
ct0=pd.crosstab(base_TCGA['Age'],base_TCGA['SOX9-AS1_expression'])
ct0

SOX9-AS1_expression,High,Low
Age,,
1,161,128
2,372,405


In [131]:
ct1=pd.crosstab(base_TCGA['Histopathology'],base_TCGA['SOX9-AS1_expression'])
ct1

SOX9-AS1_expression,High,Low
Histopathology,,
Infiltrating Ductal Carcinoma,372,387
Infiltrating Lobular Carcinoma,116,85
Mixed Histology (NOS),12,17
Other,33,44


In [132]:
ct2=pd.crosstab(base_TCGA['Subtype PAM50'],base_TCGA['SOX9-AS1_expression'])
ct2

SOX9-AS1_expression,High,Low
Subtype PAM50,,
BRCA_Basal,163,8
BRCA_Her2,26,51
BRCA_LumA,233,265
BRCA_LumB,35,162


In [133]:
ct3=pd.crosstab(base_TCGA['Radiation Therapy'],base_TCGA['SOX9-AS1_expression'])
ct3

SOX9-AS1_expression,High,Low
Radiation Therapy,,
No,207,217
Yes,270,272


In [121]:
#Importing relevant libraries to perform calculations
from scipy.stats import chi2_contingency
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests

The author of the below custom functions is Moran Neuhof, and the wrapper (automated) code is available here: https://github.com/neuhofmo/chisq_test_wrapper.

In [122]:
#First function
def get_asterisks_for_pval(p_val):
    """Receives the p-value and returns asterisks string."""
    if p_val > 0.05:
        p_text = "ns"  # above threshold => not significant
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text

In [123]:
#Second function
def chisq_and_posthoc_corrected(df):
    """Receives a crosstab dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after Bonferroni correction)
    Of note, you can change the correction method. The original code runs with FDR correction."""
   
# start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(df, correction=True)
    print(f"Chi2 result of the contingency table: {chi2}, p-value: {p}")
    
    # post-hoc
    all_combinations = list(combinations(df.index, 2))  # gathering all combinations for post-hoc chi2
    p_vals = []
    print("Significance results:")
    for comb in all_combinations:
        new_df = df[(df.index == comb[0]) | (df.index == comb[1])]
        chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
        p_vals.append(p)
        # print(f"For {comb}: {p}")  # uncorrected

    # checking significance
    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method='bonferroni')[:2] #you can use the method of preference. 
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print(f"{comb}: p_value: {p_val:5f}; corrected: {corr_p_val:5f} ({get_asterisks_for_pval(p_val)}) reject: {reject}")

In [143]:
#Age range versus SOX9-AS1 expression
chisq_and_posthoc_corrected(ct0)

Chi2 result of the contingency table: 4.861141913045027, p-value: 0.027468149998195853
Significance results:
(1, 2): p_value: 0.027468; corrected: 0.027468 (*) reject: True


In [134]:
#Histological type versus SOX9-AS1 expression
chisq_and_posthoc_corrected(ct1)

Chi2 result of the contingency table: 7.511034752056032, p-value: 0.057275598857151014
Significance results:
('Infiltrating Ductal Carcinoma', 'Infiltrating Lobular Carcinoma'): p_value: 0.034486; corrected: 0.206919 (*) reject: False
('Infiltrating Ductal Carcinoma', 'Mixed Histology (NOS)'): p_value: 0.536726; corrected: 1.000000 (ns) reject: False
('Infiltrating Ductal Carcinoma', 'Other'): p_value: 0.362801; corrected: 1.000000 (ns) reject: False
('Infiltrating Lobular Carcinoma', 'Mixed Histology (NOS)'): p_value: 0.145647; corrected: 0.873880 (ns) reject: False
('Infiltrating Lobular Carcinoma', 'Other'): p_value: 0.036792; corrected: 0.220750 (*) reject: False
('Mixed Histology (NOS)', 'Other'): p_value: 1.000000; corrected: 1.000000 (ns) reject: False


In [135]:
#PAM50 subtype versus SOX9-AS1 expression
chisq_and_posthoc_corrected(ct2)

Chi2 result of the contingency table: 231.87073577613586, p-value: 5.4492098664699905e-50
Significance results:
('BRCA_Basal', 'BRCA_Her2'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('BRCA_Basal', 'BRCA_LumA'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('BRCA_Basal', 'BRCA_LumB'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('BRCA_Her2', 'BRCA_LumA'): p_value: 0.043996; corrected: 0.263979 (*) reject: False
('BRCA_Her2', 'BRCA_LumB'): p_value: 0.006932; corrected: 0.041593 (**) reject: True
('BRCA_LumA', 'BRCA_LumB'): p_value: 0.000000; corrected: 0.000000 (****) reject: True


In [136]:
#Radiation therapy versus SOX9-AS1 expression
chisq_and_posthoc_corrected(ct3)

Chi2 result of the contingency table: 0.058583619252556535, p-value: 0.8087487565771619
Significance results:
('No', 'Yes'): p_value: 0.808749; corrected: 0.808749 (ns) reject: False


My colleagues had already dummy coded the clinical stage variable, setting NANs into 5. I will filter these values and save them in another dataframe in order to analyze them.

In [144]:
base_TCGA2=base_TCGA[base_TCGA['Clinical Stage']<5.0]

In [145]:
#1 to 4 categories, without the 5th one
base_TCGA2['Clinical Stage'].value_counts()

2    604
3    245
1    180
4     18
Name: Clinical Stage, dtype: int64

In [146]:
ct4=pd.crosstab(base_TCGA2['Clinical Stage'],base_TCGA2['SOX9-AS1_expression'])
ct4

SOX9-AS1_expression,High,Low
Clinical Stage,,
1,91,89
2,313,291
3,112,133
4,9,9


In [148]:
#clinical stage versus SOX9-AS1 expression
chisq_and_posthoc_corrected(ct4)

Chi2 result of the contingency table: 2.614972206212673, p-value: 0.45487076973878204
Significance results:
(1, 2): p_value: 0.831122; corrected: 1.000000 (ns) reject: False
(1, 3): p_value: 0.373986; corrected: 1.000000 (ns) reject: False
(1, 4): p_value: 1.000000; corrected: 1.000000 (ns) reject: False
(2, 3): p_value: 0.124354; corrected: 0.746122 (ns) reject: False
(2, 4): p_value: 1.000000; corrected: 1.000000 (ns) reject: False
(3, 4): p_value: 0.914690; corrected: 1.000000 (ns) reject: False


I will now perform the analysis using the METABRIC cohort.

In [71]:
metabric=pd.read_csv('METABRIC.csv')

In [160]:
metabric

,Age at Diagnosis,Categ,Cancer Type Detailed,Categ.1,Chemotherapy,Categ.2,ER status measured by IHC,Unnamed: 7,ER Status,Categ.3,...,Tumor Stage,Categ.8,SOX9-AS1: mRNA expression (microarray),Level expression,Level categoric,Pam50_Claudin-low subtype,Unnamed: 24,3-Gene classifier subtype,BRCA classifier subtype,Unnamed: 27
0,"21,93",1,Breast Invasive Ductal Carcinoma,1,YES,1,Negative,NaN,Negative,2,...,3.0,3,"6,973637",High,1,Excluded samples,5,ER-/HER2-,TNBC,4
1,"26,36",1,Breast Invasive Ductal Carcinoma,1,NO,2,Negative,NaN,Negative,2,...,2.0,2,"6,705706",High,1,Basal,4,ER-/HER2-,TNBC,4
2,"28,62",1,Breast Invasive Ductal Carcinoma,1,NO,2,Negative,NaN,Negative,2,...,2.0,2,"6,530297",High,1,Basal,4,ER-/HER2-,TNBC,4
3,"28,96",1,Breast Invasive Ductal Carcinoma,1,NO,2,Negative,NaN,Negative,2,...,NaN,5,"6,516583",High,1,Basal,4,ER-/HER2-,TNBC,4
4,"29,92",1,Breast Invasive Ductal Carcinoma,1,NO,2,Negative,NaN,Negative,2,...,1.0,1,"6,515348",High,1,Basal,4,ER-/HER2-,TNBC,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,"86,24",2,Breast Invasive Ductal Carcinoma,1,YES,1,Positve,NaN,Positive,1,...,2.0,2,"4,99293",Low,2,LumB,2,HER2+,HER2,3
1696,"87,32",2,Breast Invasive Ductal Carcinoma,1,YES,1,Positve,NaN,Positive,1,...,3.0,3,"4,968883",Low,2,LumB,2,HER2+,HER2,3
1697,"88,8",2,Breast Invasive Ductal Carcinoma,1,YES,1,Positve,NaN,Positive,1,...,1.0,1,"4,961279",Low,2,Excluded samples,5,HER2+,HER2,3
1698,"90,23",2,Breast Invasive Ductal Carcinoma,1,YES,1,Positve,NaN,Negative,2,...,1.0,1,"4,922177",Low,2,Excluded samples,5,HER2+,HER2,3


In [170]:
metabric.dtypes


Age at Diagnosis                           object
Categ                                       int64
Cancer Type Detailed                       object
Categ.1                                     int64
Chemotherapy                               object
Categ.2                                     int64
ER status measured by IHC                  object
Unnamed: 7                                float64
ER Status                                  object
Categ.3                                     int64
PR Status                                  object
Categ.4                                     int64
HER2 Status                                object
Categ.5                                     int64
Neoplasm Histologic Grade                 float64
Categ.6                                     int64
Radio Therapy                              object
Categ.7                                     int64
Tumor Stage                               float64
Categ.8                                     int64


In [72]:
metabric['Pam50_Claudin-low subtype'].isnull().sum()

0

In [73]:
metabric['BRCA classifier subtype'].isnull().sum()

0

In [74]:
metabric['Cancer Type Detailed'].isnull().sum()

0

In [75]:
metabric['Neoplasm Histologic Grade'].isnull().sum()

63

In [77]:
metabric['Tumor Stage'].isnull().sum()

457

In [76]:
#With NaNs
pd.crosstab(metabric['Neoplasm Histologic Grade'].fillna('missing'),metabric['Level expression'])


Level expression,High,Low
Neoplasm Histologic Grade,,
1.0,71,78
2.0,309,361
3.0,434,384
missing,36,27


In [78]:
#With NaNs
pd.crosstab(metabric['Tumor Stage'].fillna('missing'),metabric['Level expression'])

Level expression,High,Low
Tumor Stage,,
1.0,217,213
2.0,347,355
3.0,48,55
4.0,1,7
missing,237,220


In [177]:
ctm0= pd.crosstab(metabric['Tumor Stage'],metabric['Level expression'])
ctm0

Level expression,High,Low
Tumor Stage,,
1.0,217,213
2.0,347,355
3.0,48,55
4.0,1,7


In [178]:
ctm1=pd.crosstab(metabric['Neoplasm Histologic Grade'],metabric['Level expression'])
ctm1

Level expression,High,Low
Neoplasm Histologic Grade,,
1.0,71,78
2.0,309,361
3.0,434,384


In [179]:
ctm2=pd.crosstab(metabric['BRCA classifier subtype'],metabric['Level expression'])
ctm2

Level expression,High,Low
BRCA classifier subtype,,
HER2,129,59
Luminal A,212,407
Luminal B,230,373
TNBC,279,11


In [181]:
ctm4=pd.crosstab(metabric['Cancer Type Detailed'],metabric['Level expression'])
ctm4

Level expression,High,Low
Cancer Type Detailed,,
Breast Invasive Ductal Carcinoma,627,700
Breast Invasive Lobular Carcinoma,79,51
Mixed Histology,126,88
Other,18,11


In [182]:
#Tumor stage vs SOX9-AS1 expression
chisq_and_posthoc_corrected(ctm0)

Chi2 result of the contingency table: 4.872514937025567, p-value: 0.18137380929310915
Significance results:
(1.0, 2.0): p_value: 0.781962; corrected: 1.000000 (ns) reject: False
(1.0, 3.0): p_value: 0.552099; corrected: 1.000000 (ns) reject: False
(1.0, 4.0): p_value: 0.076541; corrected: 0.459245 (ns) reject: False
(2.0, 3.0): p_value: 0.666722; corrected: 1.000000 (ns) reject: False
(2.0, 4.0): p_value: 0.085059; corrected: 0.510355 (ns) reject: False
(3.0, 4.0): p_value: 0.133204; corrected: 0.799223 (ns) reject: False


In [183]:
#Tumor histological grade vs SOX9-AS1 expression
chisq_and_posthoc_corrected(ctm1)

Chi2 result of the contingency table: 7.371656736546984, p-value: 0.025076393628507086
Significance results:
(1.0, 2.0): p_value: 0.803931; corrected: 1.000000 (ns) reject: False
(1.0, 3.0): p_value: 0.260288; corrected: 0.780864 (ns) reject: False
(2.0, 3.0): p_value: 0.009041; corrected: 0.027124 (**) reject: True


In [184]:
#BRCA classifier subtype vs SOX9-AS1 expression
chisq_and_posthoc_corrected(ctm2)

Chi2 result of the contingency table: 369.0746268039517, p-value: 1.1044147718111227e-79
Significance results:
('HER2', 'Luminal A'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('HER2', 'Luminal B'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('HER2', 'TNBC'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('Luminal A', 'Luminal B'): p_value: 0.174855; corrected: 1.000000 (ns) reject: False
('Luminal A', 'TNBC'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('Luminal B', 'TNBC'): p_value: 0.000000; corrected: 0.000000 (****) reject: True


In [186]:
#Hystology vs SOX9-AS1 expression
chisq_and_posthoc_corrected(ctm4)

Chi2 result of the contingency table: 18.48391312414028, p-value: 0.0003494925596352881
Significance results:
('Breast Invasive Ductal Carcinoma', 'Breast Invasive Lobular Carcinoma'): p_value: 0.004349; corrected: 0.026092 (**) reject: True
('Breast Invasive Ductal Carcinoma', 'Mixed Histology'): p_value: 0.002039; corrected: 0.012236 (**) reject: True
('Breast Invasive Ductal Carcinoma', 'Other'): p_value: 0.163653; corrected: 0.981919 (ns) reject: False
('Breast Invasive Lobular Carcinoma', 'Mixed Histology'): p_value: 0.815610; corrected: 1.000000 (ns) reject: False
('Breast Invasive Lobular Carcinoma', 'Other'): p_value: 1.000000; corrected: 1.000000 (ns) reject: False
('Mixed Histology', 'Other'): p_value: 0.899112; corrected: 1.000000 (ns) reject: False


In [191]:
metabric2= metabric[metabric['Pam50_Claudin-low subtype']!= 'Excluded samples']

In [192]:
metabric2['Pam50_Claudin-low subtype'].value_counts()

LumA     631
LumB     412
Her2     190
Basal    161
Name: Pam50_Claudin-low subtype, dtype: int64

In [193]:
ctm3=pd.crosstab(metabric2['Pam50_Claudin-low subtype'],metabric2['Level expression'])
ctm3

Level expression,High,Low
Pam50_Claudin-low subtype,,
Basal,128,33
Her2,106,84
LumA,274,357
LumB,173,239


In [194]:
#PAM50 Claudin low classifier subtype vs SOX9-AS1 expression
chisq_and_posthoc_corrected(ctm3)

Chi2 result of the contingency table: 79.40093975075186, p-value: 4.1259860582474054e-17
Significance results:
('Basal', 'Her2'): p_value: 0.000005; corrected: 0.000028 (****) reject: True
('Basal', 'LumA'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('Basal', 'LumB'): p_value: 0.000000; corrected: 0.000000 (****) reject: True
('Her2', 'LumA'): p_value: 0.003568; corrected: 0.021406 (**) reject: True
('Her2', 'LumB'): p_value: 0.002158; corrected: 0.012947 (**) reject: True
('LumA', 'LumB'): p_value: 0.694230; corrected: 1.000000 (ns) reject: False


Results are  shown in tables 2 and 3.